In [ ]:
#| default_exp pymor.timestepping

# pymor.timestepping

> Extended functionality for [pyMOR](https://pymor.org/) time steppers

In [ ]:
#| export
from fastcore.basics import patch
import ipywidgets as widgets

import numpy as np
from scikits.odes.sundials import cvode
import pymor
from pymor.algorithms.to_matrix import to_matrix
from pymor.algorithms.timestepping import TimeStepper

from pylgs.utilities.sparse import sparse2d_identity, spilu, restrict_bandwidth
from pylgs.pymor.vectorarrays import *
from pylgs.pymor.operators import *

In [ ]:
#| export
try:
    @pymor.defaults('cvode_bdf_rtol', 'cvode_bdf_atol', 'cvode_bdf_max_steps', 'cvode_stiff_switching_t_switch')
    def cvode_solver_options(
        cvode_bdf_rtol=1e-3,
        cvode_bdf_atol=1e-6,
        cvode_bdf_max_steps=1000,
        cvode_bdf_inflection_times=None,
        cvode_stiff_switching_t_switch=1e-7
    ):
        opts = {
            'cvode_bdf': {'type': 'cvode_bdf', 'atol': cvode_bdf_atol, 'rtol': cvode_bdf_rtol, 'max_steps': cvode_bdf_max_steps, 'inflection_times': cvode_bdf_inflection_times},
            'cvode_stiff_switching': {'type': 'cvode_stiff_switching', 't_switch': cvode_stiff_switching_t_switch}
        }
        return opts
except ValueError: pass

### solve -


In [ ]:
#| export
@patch
def solve(self:TimeStepper, initial_time, end_time, initial_data, operator, rhs=None, mass=None, mu=None, num_values=None):
    """Apply time-stepper to the equation.

    The equation is of the form ::

        M(mu) * d_t u + A(u, mu, t) = F(mu, t),
                         u(mu, t_0) = u_0(mu).

    Parameters
    ----------
    initial_time
        The time at which to begin time-stepping.
    end_time
        The time until which to perform time-stepping.
    initial_data
        The solution vector at initial_timeinitial_time.
    operator
        The |Operator| A.
    rhs
        The right-hand side F (either |VectorArray| of length 1 or |Operator| with
        source.dim == 1source.dim == 1). If NoneNone, zero right-hand side is assumed.
    mass
        The |Operator| M. If NoneNone, the identity operator is assumed.
    mu
        |Parameter values| for which operatoroperator and rhsrhs are evaluated. The current
        time is added to mumu with key tt.
    num_values
        The number of returned vectors of the solution trajectory. If NoneNone, each
        intermediate vector that is calculated is returned.

    Returns
    -------
    |VectorArray| containing the solution trajectory.
    """
    try:
        num_time_steps = self.estimate_time_step_count(initial_time, end_time)
    except NotImplementedError:
        num_time_steps = 0
    iterator = self.iterate(initial_time, end_time, initial_data, operator, rhs=rhs, mass=mass, mu=mu,
                            num_values=num_values)
    iterator = list(iterator)
    if isinstance(iterator[0], XarrayVectorArray):
        return iterator[0]
    U = operator.source.empty(reserve=num_values if num_values else num_time_steps + 1)
    t = []
    for U_n, t_n in iterator:
        U.append(U_n)
        t.append(t_n)
    try: return operator.source.from_numpy(U.to_numpy(), l={"Time": t})
    except: return U

In [ ]:
#| export
class AdamsTimeStepper(TimeStepper):
    def __init__(self):
        self.__auto_init(locals())
        
    def iterate(self, initial_time, end_time, initial_data, operator, rhs=None, mass=None, mu=None, num_values=None):
        a = operator.assemble(mu.with_(t=0.)).matrix
        b = rhs.to_numpy().ravel()
        
        def cvode_rhs(t, y, ydot):
            np.copyto(ydot, (-a.dot(y) + b))
        
        self._solver = cvode.CVODE(cvode_rhs, lmm_type='Adams', nonlinsolver='fixedpoint', max_steps=1000000, one_step_compute=num_values is None)
        if num_values is not None:
            self._t_list = np.linspace(initial_time, end_time, num_values)
            sol = self._solver.solve(self._t_list, initial_data.to_numpy()[0])
            y = sol.values.y
        else:
            self._t_list = [initial_time]
            y = list(initial_data.to_numpy())
            self._solver.init_step(t0=self._t_list[0], y0=y[0])
            while self._t_list[-1] < end_time:
                sol = self._solver.step(t=end_time)
                self._t_list.append(sol.values.t)
                y.append(sol.values.y)
        return ((operator.range.from_numpy(u), t) for u, t in zip(y, self._t_list))

In [ ]:
#| export
class BDFTimeStepper(TimeStepper):
    def __init__(self):
        # self.__auto_init(locals())
        pass
        
    def iterate(self, initial_time, end_time, initial_data, operator, rhs=None, mass=None, mu=None, num_values=None, solver_options=None):
        options = cvode_solver_options()['cvode_bdf']
        if solver_options:
            options.update(solver_options)

        progress = widgets.FloatProgress(
            value=initial_time,
            min=initial_time,
            max=end_time,
            bar_style='info',
            orientation='horizontal'
        )
        display(progress)
        
        def cvode_rhs(t, y, ydot):
            progress.value = t
            np.copyto(ydot, (-operator.assemble(mu.with_(t=t)).apply(operator.source.from_numpy(y)) + rhs.as_range_array(mu.with_(t=t))).to_numpy()[0])

        def preconditioner_setup(t, y, jok, jcurPtr, gamma, user_data):
            """Generate P and do ILU decomposition."""
            if jok:
                jcurPtr.value = False
            else:
                user_data['approximate_jacobian'] = -to_matrix(operator.assemble(mu.with_(t=t)))
                user_data['approximate_jacobian'] = restrict_bandwidth(user_data['approximate_jacobian'], operator.solver_options['inverse']['preconditioner_bandwidth'])
                jcurPtr.value = True
            # Scale jacobian by -gamma, add identity matrix and do LU decomposition
            p = -gamma*user_data['approximate_jacobian'] + sparse2d_identity(user_data['approximate_jacobian'].shape[0])
            user_data['factored_preconditioner'] = spilu(p.tocsc()) # , permc_spec='NATURAL')
            return 0

        def preconditioner_solve(t, y, r, z, gamma, delta, lr, user_data):
            """ Solve the block-diagonal system Pz = r. """
            np.copyto(z, user_data['factored_preconditioner'].solve(r))
            return 0              
        
        self._solver = cvode.CVODE(
            cvode_rhs,
            lmm_type='BDF', 
            nonlinsolver='newton', 
            linsolver='spgmr',
            precond_type='left',
            prec_setupfn=preconditioner_setup, 
            prec_solvefn=preconditioner_solve,
            rtol=options['rtol'], 
            atol=options['atol'], 
            max_steps=options['max_steps'],
            user_data={}
        )
        t_list = np.linspace(initial_time, end_time, num_values)
        sol = self._solver.solve(t_list, initial_data.to_numpy()[0])
        progress.close()
        if isinstance(operator.source, XarrayVectorSpace):
            return [operator.source.from_numpy(sol.values.y, extended_dim={'Time': t_list})]
        return ((operator.source.from_numpy(u), t) for u, t in zip(sol.values.y, t_list))

In [ ]:
from pymor.basic import *

In [ ]:
p = thermal_block_problem([2,2])
m, _ = discretize_stationary_cg(p)
pp = InstationaryProblem(p, initial_data=ConstantFunction(0., 2), T=1.)
mm, _ = discretize_instationary_cg(pp, nt=10)

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [ ]:
solver_options={'inverse': {'type': 'scipy_lgmres_spilu', 'preconditioner_bandwidth': 2}}

In [ ]:
mm = mm.with_(time_stepper=BDFTimeStepper(), num_values=200, operator=mm.operator.with_(solver_options=solver_options))

In [ ]:
mm.solve({'diffusion': [.5, .6, .7, .8]})

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

FloatProgress(value=0.0, bar_style='info', max=1.0)

NumpyVectorArray(
    NumpyVectorSpace(20201, id='STATE'),
    [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
      0.00000000e+00 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.67496868e-07
      1.67496868e-07 1.65840032e-07]
     [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.34985394e-07
      3.34985394e-07 3.30310367e-07]
     ...
     [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.65871350e-05
      2.59707286e-05 1.76848884e-05]
     [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.66890353e-05
      2.60647313e-05 1.77289256e-05]
     [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.67906338e-05
      2.61583773e-05 1.77726439e-05]],
    _len=200)

In [ ]:
#| hide
# import matplotlib.pyplot as plt
# import numpy as np
# import scipy.sparse as sps
# from pymor.models.iosys import LTIModel
# import numpy as np
# import scipy.sparse as sps
# from pymor.core.logger import set_log_levels
# from pymor.algorithms.timestepping import ImplicitEulerTimeStepper

# set_log_levels({'pymor': 'WARNING'})

# k = 50
# n = 2 * k + 1

# E = sps.eye(n, format='lil')
# E[0, 0] = E[-1, -1] = 0.5
# E = E.tocsc()

# d0 = n * [-2 * (n - 1)**2]
# d1 = (n - 1) * [(n - 1)**2]
# A = sps.diags([d1, d0, d1], [-1, 0, 1], format='lil')
# A[0, 0] = A[-1, -1] = -n * (n - 1)
# A = A.tocsc()

# B = np.zeros((n, 2))
# B[:, 0] = 1
# B[0, 0] = B[-1, 0] = 0.5
# B[0, 1] = n - 1

# C = np.zeros((3, n))
# C[0, :n//3] = C[1, n//3:2*n//3] = C[2, 2*n//3:] = 1
# C /= C.sum(axis=1)[:, np.newaxis]

# fom = LTIModel.from_matrices(A, B, C, E=E)

# fom = fom.with_(T=4, time_stepper=ImplicitEulerTimeStepper(200))

# fom = fom.with_(solver_options=solver_options)

# sol = fom.solve(input='[sin(4 * t[0]), sin(6 * t[0])]')
# Y = fom.C.apply(sol).to_numpy()

# # Y = fom.output(input='[sin(4 * t[0]), sin(6 * t[0])]')

# fig, ax = plt.subplots()
# for i, y in enumerate(Y.T):
#     ax.plot(np.linspace(0, fom.T, fom.time_stepper.nt + 1), y, label=f'$y_{i+1}(t)$')
# _ = ax.set(xlabel='$t$', ylabel='$y(t)$', title='Output')
# _ = ax.legend()

# fom = fom.with_(
#     T=4, time_stepper=BDFTimeStepper(), num_values=200, 
#     A=fom.A.with_(solver_options=solver_options), 
#     B=fom.B.with_(solver_options=solver_options),
#     C=fom.C.with_(solver_options=solver_options),
#     E=fom.E.with_(solver_options=solver_options)
# )

# fom

# sol = fom.solve(input='[sin(4 * t[0]), sin(6 * t[0])]')
# Y = fom.C.apply(sol).to_numpy()

# # Y = fom.output(input='[sin(4 * t[0]), sin(6 * t[0])]')

# fig, ax = plt.subplots()
# for i, y in enumerate(Y.T):
#     ax.plot(np.linspace(0, fom.T, fom.num_values), y, label=f'$y_{i+1}(t)$')
# _ = ax.set(xlabel='$t$', ylabel='$y(t)$', title='Output')
# _ = ax.legend()

In [ ]:
#| export
class StiffSwitchingTimeStepper(TimeStepper):
    def __init__(self):
        # self.__auto_init(locals())
        pass
        
    def iterate(self, initial_time, end_time, initial_data, operator, rhs=None, mass=None, mu=None, num_values=None, solver_options=None):
        options = cvode_solver_options()['cvode_stiff_switching']
        if solver_options:
            options.update(solver_options)

        if initial_time < options['t_switch']:
            nonstiff_solver = AdamsTimeStepper()
            nonstiff = list(nonstiff_solver.iterate(initial_time, options['t_switch'], initial_data, operator, rhs=rhs, mass=mass, mu=mu, num_values=num_values))
            nonstiff_t_list = nonstiff_solver._t_list
            initial_data = nonstiff[-1][0]
            initial_time = nonstiff_t_list[-1]
        else:
            nonstiff = []
            nonstiff_t_list = []

        if end_time > options['t_switch']:
            stiff_solver = BDFTimeStepper()
            stiff = list(stiff_solver.iterate(options['t_switch'], end_time, initial_data, operator, rhs=rhs, mass=mass, mu=mu, num_values=num_values, solver_options=solver_options))
            stiff_t_list = stiff_solver._t_list            
        else:
            stiff = []
            stiff_t_list = []

        self._t_list = np.concatenate([nonstiff_t_list, stiff_t_list])
        
        return nonstiff + stiff

## Export - 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()